In [1]:
# This notebook uses the python2 kernel.

import matlab.engine

import os
os.chdir('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/Code/Library')
# Need to have the TFOCS package in Library to call the slope solver.
# Link to TFOCS: https://github.com/cvxr/TFOCS.git
# Version 1.3, October 10, 2013

from math import log, exp
import numpy as np

from scipy import stats

from collections import namedtuple

from copy import deepcopy

import matplotlib.pyplot as plt

from ExperimentFunc import exp_func, beta_gen_slope
from Step2 import find_v_slope
from Step3 import solve_omega, gw_l1, proj_l1_tan_cone, proj_l1_neg_tan_cone

In [2]:
eng = matlab.engine.start_matlab()

matpath = eng.genpath('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/Code/Library/TFOCS')
eng.addpath(matpath)

'/Users/yufei/Documents/MATLAB:/Applications/MATLAB_R2019a.app/toolbox/matlab/capabilities:/Applications/MATLAB_R2019a.app/toolbox/matlab/datafun:/Applications/MATLAB_R2019a.app/toolbox/matlab/datatypes:/Applications/MATLAB_R2019a.app/toolbox/matlab/elfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/elmat:/Applications/MATLAB_R2019a.app/toolbox/matlab/funfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/general:/Applications/MATLAB_R2019a.app/toolbox/matlab/iofun:/Applications/MATLAB_R2019a.app/toolbox/matlab/lang:/Applications/MATLAB_R2019a.app/toolbox/matlab/matfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/mvm:/Applications/MATLAB_R2019a.app/toolbox/matlab/ops:/Applications/MATLAB_R2019a.app/toolbox/matlab/polyfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/randfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/sparfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/specfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/strfun:/Applications/MATLAB_R2019a.app/toolbox/matlab/

In [3]:
def solve_beta_slope(X, Y, param_set):
    """
    Solve the SLOPE by calling a matlab function in TFOCS package.
    
    @param X np.array(n, p): the data matrix.
    @param Y np.array(n, ): the response vector.
    @param param_set list[np.array(p, )]: contains an array of tuning param according to the 'slope meets lasso' paper
    
    @return np.array(p, ): the coefficient got by SLOPE.
    """
    Y_t = eng.transpose(matlab.double(Y.tolist()))
    lbd_t = eng.transpose(matlab.double(param_set[0].tolist()))
    beta_hat = eng.solver_SLOPE(matlab.double(X.tolist()), Y_t, lbd_t)
    return np.asarray(beta_hat).squeeze()

In [4]:
Params = namedtuple('Params', ['step1', 'step2', 'step3'])

### <span style="color:purple">1) Cov(X) = I</span>

In [87]:
N = 100
n = 1000
p = 1000
Sigma_sqrt = np.eye(p)
noise_sd = 2
debias_idx = p - 1

cardi = 0.005   # number of non-zero coordinates
su = int(p * cardi)        # upper bound of number of non-zero coordinates

# construct lambda vector
A = (80) * 2*(4+2**0.5)                # the constant used to form lambda in constraint. Tuned. Don't touch! The estimation of sigma is super sensitive about this.
lbd_vec = A * noise_sd * (np.log(2*p/np.arange(1,p+1))/n)**0.5

C = (su * log(exp(1)*p/su) / n**0.5)**(-1/3)   # the constant in the bound of L2 error

param_set = Params([lbd_vec], 
                   [C, su], 
                   [gw_l1, proj_l1_tan_cone, proj_l1_neg_tan_cone])

In [88]:
z2, z_biased2 = exp_func(N,
             n,
             p, 
             Sigma_sqrt, 
             noise_sd, 
             debias_idx,
             param_set, 
             beta_gen_slope, 
             solve_beta_slope, 
             find_v_slope, 
             solve_omega)

('iter:', 0)
('The L2 error: ', 0.7034854163371727)
('iter:', 1)
('The L2 error: ', 0.5449491746694153)
('iter:', 2)
('The L2 error: ', 0.6379773362803635)
('iter:', 3)
('The L2 error: ', 0.5942290366852483)
('iter:', 4)
('The L2 error: ', 0.5644181353613701)
('iter:', 5)
('The L2 error: ', 0.653874971798808)
('iter:', 6)
('The L2 error: ', 0.7166011127487477)
('iter:', 7)
('The L2 error: ', 0.7596390120278274)
('iter:', 8)
('The L2 error: ', 0.6446911135197673)
('iter:', 9)
('The L2 error: ', 0.5827467088021657)
('iter:', 10)
('The L2 error: ', 0.637655099390062)
('iter:', 11)
('The L2 error: ', 0.5634368276846595)
('iter:', 12)
('The L2 error: ', 0.8002506198997914)
('iter:', 13)
('The L2 error: ', 0.5715872100011637)
('iter:', 14)
('The L2 error: ', 0.6663760801974378)
('iter:', 15)
('The L2 error: ', 0.6913263378055698)
('iter:', 16)
('The L2 error: ', 0.7348315992828509)
('iter:', 17)
('The L2 error: ', 0.6644304414280302)
('iter:', 18)
('The L2 error: ', 0.7626381926489672)
('ite

#### Compare the mean of the (debiased_beta - beta) and (non-debiased_beta - beta)

In [89]:
mean_non_debiased = np.mean(z_biased2)
print("The mean of (non_debiased_beta - beta) is: ", mean_non_debiased)

('The mean of (non_debiased_beta - beta) is: ', 3.732388464794405)


In [90]:
mean_debiased = np.mean(np.array(z2))
print("The mean of (debiased_beta - beta) is: ", mean_debiased)

('The mean of (debiased_beta - beta) is: ', -0.3417278451923971)


#### Save the simulation results

In [91]:
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/identity_z_biased.npy', z_biased2)
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/identity_z.npy', z2)

###  <span style="color:purple">2) Cov(X) with bounded eigenvalues</span>

In [82]:
# other parameters are the same as cov=I case

# Generate a cov matrix with bounded eigenvalues
# generate eigenvalues
cov_eigv = np.random.uniform(low = 0.5, high = 3.0, size = (p,))
D_sqrt = np.diag(cov_eigv**0.5)
# generate an orthonormal matrix
a = np.random.normal(size = (p,p))
u, s, vh = np.linalg.svd(np.matmul(a.T,a), full_matrices=True)
# generate the square root of cov matrix 
Sigma_sqrt = np.matmul(D_sqrt,u.T)

In [83]:
z, z_biased = exp_func(N,
             n,
             p, 
             Sigma_sqrt, 
             noise_sd, 
             debias_idx,
             param_set, 
             beta_gen_slope, 
             solve_beta_slope, 
             find_v_slope, 
             solve_omega)

('iter:', 0)
('The L2 error: ', 0.37506717438600556)
('iter:', 1)
('The L2 error: ', 0.5244401712664486)
('iter:', 2)
('The L2 error: ', 0.45920989215292224)
('iter:', 3)
('The L2 error: ', 0.4803370440441076)
('iter:', 4)
('The L2 error: ', 0.5003708999361449)
('iter:', 5)
('The L2 error: ', 0.5105583328656228)
('iter:', 6)
('The L2 error: ', 0.47511712066508593)
('iter:', 7)
('The L2 error: ', 0.48753951840646903)
('iter:', 8)
('The L2 error: ', 0.4486124728050898)
('iter:', 9)
('The L2 error: ', 0.38264897288319466)
('iter:', 10)
('The L2 error: ', 0.325415741566226)
('iter:', 11)
('The L2 error: ', 0.3466547682192231)
('iter:', 12)
('The L2 error: ', 0.38624111223685803)
('iter:', 13)
('The L2 error: ', 0.4409238246380599)
('iter:', 14)
('The L2 error: ', 0.4188091720119363)
('iter:', 15)
('The L2 error: ', 0.3702878298543199)
('iter:', 16)
('The L2 error: ', 0.3273596405883521)
('iter:', 17)
('The L2 error: ', 0.4869083324688285)
('iter:', 18)
('The L2 error: ', 0.4693627601277034

#### Compare the mean of the (debiased_beta - beta) and (non-debiased_beta - beta)

In [84]:
mean_non_debiased = np.mean(z_biased)
print("The mean of (non_debiased_beta - beta) is: ", mean_non_debiased)

('The mean of (non_debiased_beta - beta) is: ', 2.397840295825244)


In [85]:
mean_debiased = np.mean(np.array(z))
print("The mean of (debiased_beta - beta) is: ", mean_debiased)

('The mean of (debiased_beta - beta) is: ', -0.4166391229398444)


#### Save the simulation results

In [86]:
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/bddeig_z_biased2.npy', z_biased)
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/bddeig_z2.npy', z)

### <span style = 'color:purple'>3) Cov(X) is the Cov of AR(1) Process</span>

In [13]:
# other parameters are the same as cov=I case

# Generate the squar root of cov matrix
rho = 0.4
rho_vec = []
for i in range(p):
    rho_vec.append(rho**i)
rho_vec = np.array(rho_vec)
# The cholesky decomposition of cov == the squar root of cov
Sigma_sqrt = [rho_vec]
for i in range(1, p):
    rho_vec_shifted = np.concatenate((np.zeros(i), rho_vec[:-i]))
#     print(rho_vec_shifted)
    Sigma_sqrt.append(rho_vec_shifted * (1-rho**2)**0.5)
Sigma_sqrt = np.array(Sigma_sqrt)

In [14]:
z, z_biased = exp_func(N,
             n,
             p, 
             Sigma_sqrt, 
             noise_sd, 
             debias_idx,
             param_set, 
             beta_gen_slope, 
             solve_beta_slope, 
             find_v_slope, 
             solve_omega)

('iter:', 0)
('The L2 error: ', 0.37521731620156656)
('iter:', 1)
('The L2 error: ', 0.48114001582503996)
('iter:', 2)
('The L2 error: ', 0.4691941443173258)
('iter:', 3)
('The L2 error: ', 0.35211669262391304)
('iter:', 4)
('The L2 error: ', 0.2651915113477123)
('iter:', 5)
('The L2 error: ', 0.4364546980499729)
('iter:', 6)
('The L2 error: ', 0.4532847153097186)
('iter:', 7)
('The L2 error: ', 0.4569740959547278)
('iter:', 8)
('The L2 error: ', 0.4929651598275676)
('iter:', 9)
('The L2 error: ', 0.34487936594143825)
('iter:', 10)
('The L2 error: ', 0.41290913332436174)
('iter:', 11)
('The L2 error: ', 0.4816492322970049)
('iter:', 12)
('The L2 error: ', 0.38105705098498444)
('iter:', 13)
('The L2 error: ', 0.4594848092349639)
('iter:', 14)
('The L2 error: ', 0.4604461570775178)
('iter:', 15)
('The L2 error: ', 0.561344622761566)
('iter:', 16)
('The L2 error: ', 0.2796083130586663)
('iter:', 17)
('The L2 error: ', 0.39935323619646457)
('iter:', 18)
('The L2 error: ', 0.422585247898223

#### Compare the mean of the (debiased_beta - beta) and (non-debiased_beta - beta)

In [15]:
mean_non_debiased = np.mean(z_biased)
print("The mean of (non_debiased_beta - beta) is: ", mean_non_debiased)

('The mean of (non_debiased_beta - beta) is: ', 1.9587003912087362)


In [16]:
mean_debiased = np.mean(np.array(z))
print("The mean of (debiased_beta - beta) is: ", mean_debiased)

('The mean of (debiased_beta - beta) is: ', -0.3192269374492944)


#### Save the simulation results

In [17]:
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/SLOPE/ar1_z_biased.npy', z_biased)
np.save('/Users/yufei/Documents/2-CMU/DebiasingCvxConstrained/ExpResults/SLOPE/ar1_z.npy', z)